In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.chdir("drive/MyDrive/Colab Notebooks/KGEmb")
!ls

application.log  datasets	      models		set_env.sh
colab_run.ipynb  Lap_Emb.ipynb	      optimizers	test.py
data		 Lap_No_Rships.ipynb  requirements.txt	utils
data_loader.py	 logs		      run.py


In [3]:
!echo $(pwd)/data

/content/drive/MyDrive/Colab Notebooks/KGEmb/data


In [5]:
import numpy as np
from scipy import sparse
from sklearn.manifold import SpectralEmbedding
from scipy.spatial.distance import cdist
! pip install pyamg
! pip install fastlapmap
! pip install nmslib
# from fastlapmap import LapEigenmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from fastlapmap import LapEigenmap

In [ ]:
entities=[] # create list to store vertex IDs
num_rship=0

with open('data/YAGO3-10/train.txt', 'r') as f:
    for line in f:
        edge = line.split()
        if int(edge[1]) > num_rship:
            num_rship = int(edge[1])
        edge.remove(edge[1]) # remove relationship type
        if int(edge[0]) not in entities:
            entities.append(int(edge[0])) # add head ID
        if int(edge[1]) not in entities:
            entities.append(int(edge[1])) # add tail ID
    f.close()



#rship list of matrices. To access element: mats[r][i,j]
# mats = [sparse.lil_matrix((max(entities),max(entities)), dtype=np.int8) for _ in range(num_rship+1)]
mats = [sparse.lil_matrix((max(entities),max(entities)), dtype=np.int8).tocsr() for _ in range(num_rship+1)]

with open('data/YAGO3-10/train.txt', 'r') as f:
    for line in f:
        edge = line.split()
        row_idx = entities.index(int(edge[0])) # get index for head entity
        col_idx = entities.index(int(edge[2])) # get index for tail entity
        mats[int(edge[1])][row_idx, col_idx] = 1 # add edge in adjacency matrix
        mats[int(edge[1])][col_idx, row_idx] = 1 # undirected graph so symetric adjacency matrix
        
#Now have list of rship adjacency matrices (stored as list of sparse matrices)

# embedding = SpectralEmbedding(n_components=2, eigen_solver='amg') #%%MemoryError, 12.5GB array with dtype float64

del entities

embs = []

for i in range(len(mats)):
    # print('before:', embs)
    # embs.append(embedding.fit_transform(mats[i]).astype(np.float32))
    embs.append(LapEigenmap(mats[i], n_eigs=10, similarity='diffusion', norm_laplacian=False, M=10, metric='euclidean'))
    # print('after:', embs)
#now have list of embedding matrices (stored as list of arrays)

embs = np.asarray(embs)

def hit_at_k(r, k): #Function for measuring hit at k for each query, i.e 0 or 1
    if r <= k:
        return 1
    else:
        return 0
    
    
def evaluate(embeddings, head, rship, tail): #Function to calculate metrics for each query
    dist = cdist(embeddings[rship,head,:].reshape(1, -1), embeddings[rship,:,:]).argsort()
    # dist = np.sum((embeddings[rship] - embs[rship,head,:])**2, axis=1).argsort()
    rank = np.where(dist==tail)[1][0]
    recip_rank = 1/rank
    H_1 = hit_at_k(rank, 1)
    H_3 = hit_at_k(rank, 3)
    H_10 = hit_at_k(rank, 10)
    return rank, recip_rank, H_1, H_3, H_10
    
#counter functions to sum across loop
sum_rank = 0
sum_recip_rank = 0
sum_H_1 = 0
sum_H_3 = 0
sum_H_10 = 0
len_test = 0


with open('data/YAGO3-10/test.txt', 'r') as f:
    for line in f:
        edge = line.split()
        rank, recip_rank, H_1, H_3, H_10 = evaluate(embs, int(edge[0]), int(edge[1]), int(edge[2]))
        sum_rank += rank
        sum_recip_rank += recip_rank
        sum_H_1 += H_1
        sum_H_3 += H_3
        sum_H_10 += H_10
        
        len_test= len_test + 1
    f.close()
    
#Get the mean/average for metrics
MR = sum_rank / len_test
MRR = sum_recip_rank / len_test
H_1 = sum_H_1 / len_test
H_3 = sum_H_3 / len_test
H_10 = float(sum_H_10) / len_test

#Print results
print('MRR:', MRR, ', Hits@10:', H_10, ', Hits@3: ', H_3, ', Hits@1: ', H_1)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [ ]:
edge[2]